In [6]:
# Notebook: 4_deduplicacao_com_splink.ipynb
# ==============================================================================
# Bloco 1: CARREGAMENTO E LIMPEZA FINAL DOS DADOS
# ==============================================================================
import pandas as pd
import pathlib
import duckdb
import unidecode # Biblioteca para remover acentos
from IPython.display import display, HTML

# --- Configuração ---
PASTA_PROCESSED = pathlib.Path("dados_catalogo") / "processed"
ARQUIVO_DE_ENTRADA = PASTA_PROCESSED / 'tabela_dimensao_splink.parquet'
ARQUIVO_DE_SAIDA = PASTA_PROCESSED / 'tabela_dimensao_limpa_para_splink.parquet'

print("--- Etapa de Limpeza Final Pré-Splink ---")


--- Etapa de Limpeza Final Pré-Splink ---


In [7]:
# --- Funções Auxiliares de Limpeza ---

def normalizar_string(texto):
    """Aplica limpeza segura em um campo de texto: minúsculas, remove acentos e espaços extras."""
    if texto is None or pd.isna(texto):
        return None
    texto = str(texto).lower()
    texto = unidecode.unidecode(texto)
    return texto.strip()

def normalizar_lista_em_string(texto, separador=','):
    """Normaliza uma string que representa uma lista (ex: elenco, diretores)."""
    if texto is None or pd.isna(texto):
        return None
    # Quebra a string em uma lista de nomes
    nomes = str(texto).split(separador)
    # Limpa, normaliza e ordena cada nome
    nomes_limpos = sorted([normalizar_string(nome) for nome in nomes if nome.strip()])
    # Junta a lista de volta em uma string padronizada
    return f", ".join(nomes_limpos)

# --- Execução ---
try:
    print(f"\nCarregando dados de: '{ARQUIVO_DE_ENTRADA}'")
    df = pd.read_parquet(ARQUIVO_DE_ENTRADA)
    print(f"Dados carregados com {len(df):,} perfis únicos.")
    
    # Faz uma cópia para evitar warnings do Pandas
    df_limpo = df.copy()

    # Aplica a normalização nas colunas de texto mais importantes
    print("\n1. Normalizando campos de texto (títulos, país)...")
    df_limpo['BB_Title'] = df_limpo['BB_Title'].apply(normalizar_string)
    df_limpo['BB_Original_Title'] = df_limpo['BB_Original_Title'].apply(normalizar_string)
    df_limpo['BB_Primary_Country'] = df_limpo['BB_Primary_Country'].apply(normalizar_string)

    # Aplica a normalização nas colunas de lista (elenco e diretores)
    print("2. Padronizando e ordenando listas de elenco e diretores...")
    df_limpo['BB_Cast'] = df_limpo['BB_Cast'].apply(normalizar_lista_em_string)
    # --- CORREÇÃO APLICADA AQUI ---
    df_limpo['BB_Directors'] = df_limpo['BB_Directors'].apply(normalizar_lista_em_string)
    
    # Garante o tipo de dado correto para o ano
    print("3. Ajustando tipo de dado da coluna 'BB_Year'...")
    df_limpo['BB_Year'] = pd.to_numeric(df_limpo['BB_Year'], errors='coerce').astype('Int64')

    print("\nLimpeza concluída. Verificando uma amostra dos dados transformados:")
    
    colunas_preview = ['BB_Title', 'BB_Year', 'BB_Cast', 'BB_Directors']
    display(df_limpo[colunas_preview].head())
    
    # Salva o arquivo final e limpo
    df_limpo.to_parquet(ARQUIVO_DE_SAIDA, index=False)
    print(f"\n✔ Tabela dimensão final e limpa salva em: '{ARQUIVO_DE_SAIDA}'")

except Exception as e:
    print(f"\nOcorreu um erro durante o processo de limpeza: {e}")


Carregando dados de: 'dados_catalogo\processed\tabela_dimensao_splink.parquet'
Dados carregados com 225,117 perfis únicos.

1. Normalizando campos de texto (títulos, país)...
2. Padronizando e ordenando listas de elenco e diretores...
3. Ajustando tipo de dado da coluna 'BB_Year'...

Limpeza concluída. Verificando uma amostra dos dados transformados:


,BB_Title,BB_Year,BB_Cast,BB_Directors
0,i am not okay with this,2020,"aidan wojtak-hissong, dave theune, david bende...",jonathan entwistle
1,alan gilbert and wenzel fuchs with mozart's cl...,2018,None,alan gilbert
2,azadi,2025,"abhija sivakala, abhiram radhakrishnan, anwyn ...",joe george
3,australia v brasil | disputa do 3o lugar | fif...,2001,None,None
4,on our own,2010,None,camiel zwart



✔ Tabela dimensão final e limpa salva em: 'dados_catalogo\processed\tabela_dimensao_limpa_para_splink.parquet'
